In [1]:
import time
import os
import shutil
import sys
import h5py
import tensorflow                             as tf
import numpy                                  as np
import seaborn                                as sns
import pandas                                 as pd
import matplotlib.pyplot                      as plt
import numpy.random                           as random
from sklearn.utils                        import shuffle
from sklearn.model_selection              import train_test_split
from tqdm                                 import tqdm
from os                                   import path
import pysftp                                 as sftp
from matplotlib                 import pyplot as plt 

from tensorflow                           import keras

print("\n======================================================================================================================================")
print(" TensorFlow version: {}".format(tf.__version__))
print(" Eager execution: {}".format(tf.executing_eagerly()))


 TensorFlow version: 2.5.0
 Eager execution: True


In [2]:
WORKSPACE_PATH      = os.environ['WORKSPACE_PATH']
SurQCTFldr          = WORKSPACE_PATH + '/SurQCT/surqct/'

RatesType           = 'KExcit'
ExcitType           = 'KInel'

NNRunIdx            = 10           
PathToRunFld        = SurQCTFldr + '/../' + RatesType + '/all_temperatures_nondim/' + ExcitType + '/' 

TTran               = 10000.0

In [3]:
#===================================================================================================================================
print("\n[SurQCT]: Loading Modules and Functions ...")

sys.path.insert(0, SurQCTFldr  + '/src/Reading/')
from Reading  import read_levelsdata, read_diatdata

InputFld = PathToRunFld + '/Run_' + str(NNRunIdx) + '/'
sys.path.insert(0, InputFld)
#===================================================================================================================================


[SurQCT]: Loading Modules and Functions ...


In [4]:
#===================================================================================================================================
print("\n[SurQCT]: Keep Loading Modules and Functions...")
from SurQCT_Input import inputdata

print("\n[SurQCT]: Initializing Input ...")
InputData    = inputdata(WORKSPACE_PATH, SurQCTFldr)

Prefix                    = 'Run_'
InputData.NNRunIdx        = NNRunIdx
InputData.PathToRunFld    = InputData.PathToRunFld+'/'+Prefix+str(InputData.NNRunIdx)
InputData.PathToFigFld    = InputData.PathToRunFld+'/'+InputData.PathToFigFld
InputData.PathToParamsFld = InputData.PathToRunFld+'/'+InputData.PathToParamsFld
InputData.PathToDataFld   = InputData.PathToRunFld+'/Data/'                                                               
InputData.PathToParamsFld = InputData.PathToRunFld+'/Params/' 

# InputData.PathToHDF5File  = InputData.WORKSPACE_PATH  + '/Air_Database/HDF5_Database/N3_NASA.hdf5'
# InputData.Molecules       = ['N2','N2'] 
# InputData.PathToLevelsFile= [InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/N2_LeRoy_nd_ELog.csv',
#                              InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/N2_LeRoy_nd_ELog.csv']
# InputData.PathToDiatFile  = [InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp',
#                              InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp']   
# #InputData.PathToGrouping  = self.WORKSPACE_PATH  + '/Air_Database/Run_0D/database/grouping/O3_UMN/O2/LevelsMap_DPM45.csv'   
#===================================================================================================================================


[SurQCT]: Keep Loading Modules and Functions...

[SurQCT]: Initializing Input ...


In [5]:
#===================================================================================================================================
print("\n[SurQCT]: Loading Final Modules ... ")

sys.path.insert(0, SurQCTFldr  + '/src/Model/' + InputData.ApproxModel + '/')
from Model import model


InputData.DefineModelIntFlg = 0
InputData.TrainIntFlg       = 0
NN_KExch                    = model(InputData, InputData.PathToRunFld, None, None)
NN_KExch.load_params(InputData.PathToParamsFld)
#===================================================================================================================================


[SurQCT]: Loading Final Modules ... 
[SurQCT]:   Variables Selected for Training:
[SurQCT]:     x =  ['EVib_i', 'ERotR_i', 'EInt_i', 'VMin_i', 'VMax_i', 'rMin_i', 'rMax_i', 'ri_i', 'ro_i', 'Tau_i', 'TTran_i', 'EVib_Delta', 'ERotR_Delta', 'EInt_Delta', 'VMin_Delta', 'VMax_Delta', 'rMin_Delta', 'rMax_Delta', 'ri_Delta', 'ro_Delta', 'Tau_Delta', 'TTran_Delta']
[SurQCT]:     y =  KExcit
[SurQCT]:   Loading ML Model from Folder: /Users/sventuri/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures_nondim/KInel//Run_10/MyModel
[SurQCT]:   Restoring ML Model Weigths to File: /Users/sventuri/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures_nondim/KInel//Run_10/MyModel/MyWeights.h5
[SurQCT]:   Loading ML Model Parameters from File:  /Users/sventuri/WORKSPACE//SurQCT/surqct//../KExcit/all_temperatures_nondim/KInel//Run_10/Params//ModelCheckpoint/cp-3868.ckpt


In [6]:
Branch_KExch = keras.Model(NN_KExch.Model.inputs, NN_KExch.Model.layers[-4].output)
Branch_KExch.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 22)]              0         
_________________________________________________________________
tf.split (TFOpLambda)        [(None, 11), (None, 11)]  0         
_________________________________________________________________
normalization (Normalization (None, 11)                23        
_________________________________________________________________
Branch_HL1_i (Dense)         (None, 64)                768       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
Branch_HL2_i (Dense)         (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0     

In [7]:
Trunk_KExch = keras.Model(NN_KExch.Model.inputs, NN_KExch.Model.layers[-3].output)
Trunk_KExch.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 22)]              0         
_________________________________________________________________
tf.split (TFOpLambda)        [(None, 11), (None, 11)]  0         
_________________________________________________________________
normalization_1 (Normalizati (None, 11)                23        
_________________________________________________________________
Trunk_HL1_i (Dense)          (None, 64)                768       
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
Trunk_HL2_i (Dense)          (None, 64)                4160      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0   

In [8]:
#===================================================================================================================================
OtherVar           = InputData.OtherVar
xVarsVec_i         = InputData.xVarsVec_i + ['vqn','jqn']
xVarsVec_Delta     = InputData.xVarsVec_Delta
xVarsVec           = list(set(xVarsVec_i) | set(xVarsVec_Delta))
print('[SurQCT]:   Reading Variables: ', xVarsVec)

InputData.MultFact = 1.e+9
MinValueTrain      = 1.e-16 * InputData.MultFact
MinValueTest       = 1.e-16 * InputData.MultFact
NoiseSD            = 1.e-13 * InputData.MultFact

NMolecules         = len(InputData.PathToLevelsFile)

InputData.iLevelsVecTest = list(np.array(InputData.iLevelsVecTest) - 1)

[SurQCT]:   Reading Variables:  ['ri', 'ro', 'jqn', 'ERotR', 'VMin', 'vqn', 'rMax', 'Tau', 'EVib', 'EInt', 'VMax', 'rMin']


In [9]:
#===================================================================================================================================
### Reading Levels Info of Initial and Final Molecules
LevelsData = []
DiatData   = []
NLevels    = []
for iMol in range(NMolecules):

    LevelsDataTemp = read_levelsdata(InputData.PathToLevelsFile[iMol], xVarsVec, '')
    LevelsData.append(LevelsDataTemp)

    DiatDataTemp = read_diatdata(InputData.PathToDiatFile[iMol], InputData.Molecules[iMol], np.array([TTran]), np.array([TTran]))
    DiatData.append(DiatDataTemp)

    NLevelsTemp    = LevelsDataTemp.shape[0]
    NLevels.append(NLevelsTemp)

[SurQCT]:   Reading Molecular Levels Data from: /Users/sventuri/WORKSPACE//Air_Database/Run_0D/database/levels/O2_UMN_nd_ELog.csv
[SurQCT]:   Reading Molecular Levels Data from: /Users/sventuri/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/O2/UMN/FromUMN_Sorted.inp
[SurQCT]:   Reading Molecular Levels Data from: /Users/sventuri/WORKSPACE//Air_Database/Run_0D/database/levels/O2_UMN_nd_ELog.csv
[SurQCT]:   Reading Molecular Levels Data from: /Users/sventuri/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/O2/UMN/FromUMN_Sorted.inp


In [10]:
#=======================================================================================================================================
# Reading Excitation Rates Data 
def read_kexcitdata(InputData, PathToHDF5File, TTra, TInt, NProcTypes):
    print('[SurQCT]:       Reading HDF5 File from: ' + PathToHDF5File + ' for Excitation Rates at Temperature ' + str(int(TTra)) + 'K')

    HDF5Exist_Flg = path.exists(PathToHDF5File)
    if (HDF5Exist_Flg):
        f = h5py.File(PathToHDF5File, 'a')
    else:
        f = {'key': 'value'}

    TStr = 'T_' + str(int(TTra)) + '_' + str(int(TInt)) + '/Rates/'
    grp  = f[TStr]

    Data  = grp["Inel"]
    KInel = Data[...]

    KOther = []
    for iProc in range(2, NProcTypes):
        ExchStr    = "Exch_" + str(iProc-1)
        Data       = grp[ExchStr] 
        KOtherTemp = Data[...]
        KOther.append(KOtherTemp)

    f.close()   


    return KInel, KOther
#=======================================================================================================================================

KInelMat, KExchMatList = read_kexcitdata(InputData, InputData.PathToHDF5File, TTran, TTran, 3)

KExch = KInelMat

[SurQCT]:       Reading HDF5 File from: /Users/sventuri/WORKSPACE//Air_Database/HDF5_Database/O3_UMN.hdf5 for Excitation Rates at Temperature 10000K


In [11]:
PathToOuput = InputData.PathToRunFld + '/TrunkAndBrunch_'+InputData.Molecules[0]+'/T' + str(int(TTran)) + 'K'
try:
    os.makedirs(PathToOuput)
except OSError as e:
    pass

#===================================================================================================================================
### Opening Files for Writing Rates
Branch_File = PathToOuput + '/Branch.csv'
Trunk_File  = PathToOuput + '/Trunk.csv'
Group_File  = PathToOuput + '/Groups.csv'
NN_File     = PathToOuput + '/NN'

In [12]:
ColumnNames = []
for i in range(64):
    ColumnNames.append('N'+str(i+1))
print(ColumnNames)

['N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'N8', 'N9', 'N10', 'N11', 'N12', 'N13', 'N14', 'N15', 'N16', 'N17', 'N18', 'N19', 'N20', 'N21', 'N22', 'N23', 'N24', 'N25', 'N26', 'N27', 'N28', 'N29', 'N30', 'N31', 'N32', 'N33', 'N34', 'N35', 'N36', 'N37', 'N38', 'N39', 'N40', 'N41', 'N42', 'N43', 'N44', 'N45', 'N46', 'N47', 'N48', 'N49', 'N50', 'N51', 'N52', 'N53', 'N54', 'N55', 'N56', 'N57', 'N58', 'N59', 'N60', 'N61', 'N62', 'N63', 'N64']


In [20]:
iIdxVec              = np.array([5000])#, 2000, 3000, 4000, 4581, 5000, 6000], dtype=int)
InputData.ExoEndoFlg = False

In [21]:
### Loop on Initial States
VirginFlg = True
OtherVar_ = 'j'

#for iIdx in tqdm(range(1000, 1001), desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
Str = 'q_'+str(int(TTran))
for iIdx in tqdm(iIdxVec, desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
    iIdx -= 1
    time.sleep(0.02)
    
    if (InputData.ExoEndoFlg):
        jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] < DiatData[0]['EInt'].to_numpy()[iIdx])]
    else:
        jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    > DiatData[0][Str].to_numpy()[iIdx])]
    jNLevels              = len(jIdxVec)
    
    
    
    ### FWD Rates
    iiIdxVec              = [iIdx]*jNLevels
    
    TTranVec              = np.ones((jNLevels))*TTran
    TTranDataTemp         = pd.DataFrame({'TTran': TTranVec})
    TTranDataTemp.index   = jIdxVec

    
    iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
    iLevelsDataTemp.index = jIdxVec

    jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
    if (OtherVar == '_Delta'):
        jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
    else:
        jLevelsDataTemp   = jLevelsDataTemp
    jLevelsDataTemp.index = jIdxVec
    
    iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
    iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
    
    jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
    jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]
    
    xTemp_FWD             = pd.concat([iLevelsData, jLevelsData], axis=1)


    iLevelsDataOrig         = DiatData[0].iloc[iiIdxVec,:].copy()
    iLevelsDataOrig.index   = jIdxVec

    jLevelsDataOrig         = DiatData[1].iloc[jIdxVec,:].copy()        
    jLevelsDataOrig         = jLevelsDataOrig.subtract(jLevelsDataTemp) 
    jLevelsDataOrig.index   = jIdxVec
 
    kLevelsDataOrig         = DiatData[1].iloc[jIdxVec,:].copy()
    kLevelsDataOrig.index   = jIdxVec
        
    iLevelsDataOrig         = pd.concat([iLevelsDataOrig, TTranDataTemp], axis=1)
    iLevelsDataOrig.columns = [(VarName + '_i') for VarName in iLevelsDataOrig.columns]
    
    jLevelsDataOrig         = pd.concat([jLevelsDataOrig, TTranDataTemp], axis=1)
    jLevelsDataOrig.columns = [(VarName + OtherVar) for VarName in jLevelsDataOrig.columns]
    
    kLevelsDataOrig         = pd.concat([kLevelsDataOrig, TTranDataTemp], axis=1)
    kLevelsDataOrig.columns = [(VarName + '_j') for VarName in kLevelsDataOrig.columns]
    
    
    KExch_NN_FWD          = np.exp( NN_KExch.Model.predict(xTemp_FWD[NN_KExch.xTrainingVar]) ) / InputData.MultFact
    KExch_NN_FWD_PD       = pd.DataFrame(KExch_NN_FWD, columns=['KExch'])
    KExch_NN_FWD_PD.index = jIdxVec
    
    DataTot               = pd.concat([iLevelsDataOrig, kLevelsDataOrig, jLevelsDataOrig, KExch_NN_FWD_PD], axis=1)
    
    DataTot.to_csv(NN_File+'.csv.'+str(iIdx+1), index=False)

[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|██████████| 1/1 [00:00<00:00,  2.11it/s]


In [22]:
### Loop on Initial States
VirginFlg = True
OtherVar_ = 'j'


Str = 'q_'+str(int(TTran))
for iIdx in tqdm(iIdxVec, desc='[SurQCT]:     Evaluating TrunkNet'):
    print(iIdx)
    time.sleep(0.02)
    iIdx -= 1
    
    if (InputData.ExoEndoFlg):
        jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] < DiatData[0]['EInt'].to_numpy()[iIdx])]
    else:
        jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    > DiatData[0][Str].to_numpy()[iIdx])]
    #jIdxVec               = np.arange(NLevels[1])
    jNLevels              = len(jIdxVec)
    kIdxVec               = [iIdx]*jNLevels

    TTranVec              = np.ones((jNLevels))*TTran
    TTranDataTemp         = pd.DataFrame({'TTran': TTranVec})
    TTranDataTemp.index   = jIdxVec

    
    #########################################################################
    iLevelsDataTemp       = LevelsData[0].iloc[kIdxVec,:].copy()
    iLevelsDataTemp.index = jIdxVec

    jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
    if (OtherVar == '_Delta'):
        jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
    else:
        jLevelsDataTemp   = jLevelsDataTemp
    jLevelsDataTemp.index = jIdxVec

    iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
    iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
    jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
    jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]
    xTemp                 = pd.concat([iLevelsData, jLevelsData], axis=1)
    #########################################################################
    
    
    
    #########################################################################
    iLevelsDataOrig       = DiatData[0].iloc[kIdxVec,:].copy()
    iLevelsDataOrig.index = jIdxVec

    jLevelsDataOrig       = DiatData[1].iloc[jIdxVec,:].copy()        
    jLevelsDataOrig.index = jIdxVec
    
    kLevelsDataOrig       = DiatData[1].iloc[jIdxVec,:].copy()        
    kLevelsDataOrig       = iLevelsDataOrig.subtract(kLevelsDataOrig) 
    kLevelsDataOrig.index = jIdxVec

    iLevelsDataOrig           = pd.concat([iLevelsDataOrig, TTranDataTemp], axis=1)
    iLevelsDataOrig.columns   = [(VarName + '_i') for VarName in iLevelsDataOrig.columns]
    jLevelsDataOrig           = pd.concat([jLevelsDataOrig, TTranDataTemp], axis=1)
    jLevelsDataOrig.columns   = [(VarName + '_j') for VarName in jLevelsDataOrig.columns]   
    kLevelsDataOrig           = pd.concat([kLevelsDataOrig, TTranDataTemp], axis=1)
    kLevelsDataOrig.columns   = [(VarName + '_Delta') for VarName in kLevelsDataOrig.columns]   
    #########################################################################

    
    
    KTrunk                = Trunk_KExch.predict(xTemp[NN_KExch.xTrainingVar])
    KTrunk_PD             = pd.DataFrame(KTrunk, columns=ColumnNames)
    KTrunk_PD.index       = jIdxVec
    
    DataTot               = pd.concat([iLevelsDataOrig, jLevelsDataOrig, kLevelsDataOrig, KTrunk_PD], axis=1)
    
    if (VirginFlg):
        DataTot.to_csv(Trunk_File, index=False)
        VirginFlg = False
    else:
        DataTot.to_csv(Trunk_File, index=False, mode='a', header=False)

[SurQCT]:     Evaluating TrunkNet:   0%|          | 0/1 [00:00<?, ?it/s]

5000


[SurQCT]:     Evaluating TrunkNet: 100%|██████████| 1/1 [00:00<00:00,  1.21it/s]


In [ ]:
### Loop on Initial States
VirginFlg = True

for iIdx in tqdm(range(NLevels[0]), desc='[SurQCT]:     Evaluating BranchNet'):     
    time.sleep(0.02)
    
    jIdxVec               = [iIdx]
    jNLevels              = len(jIdxVec)
    kIdxVec               = [iIdx]*jNLevels

    TTranVec              = np.ones((jNLevels))*TTran
    TTranDataTemp         = pd.DataFrame({'TTran': TTranVec})
    TTranDataTemp.index   = jIdxVec

    
    #########################################################################
    iLevelsDataTemp       = LevelsData[0].iloc[kIdxVec,:].copy()
    iLevelsDataTemp.index = jIdxVec

    jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
    if (OtherVar == '_Delta'):
        jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
    else:
        jLevelsDataTemp   = jLevelsDataTemp
    jLevelsDataTemp.index = jIdxVec

    iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
    iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
    jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
    jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]
    xTemp                 = pd.concat([iLevelsData, jLevelsData], axis=1)
    #########################################################################
    
    
    KBranch               = Branch_KExch.predict(xTemp[NN_KExch.xTrainingVar])
    TempData              = pd.DataFrame(DiatData[0].iloc[[iIdx]])
    DataTot               = pd.concat([TempData.reset_index(drop=True), pd.DataFrame(KBranch, columns=ColumnNames)], axis=1)
    
    if (VirginFlg):
        DataTot.to_csv(Branch_File, index=False)
        VirginFlg = False
    else:
        DataTot.to_csv(Branch_File, index=False, mode='a', header=False)

[SurQCT]:     Evaluating BranchNet:  38%|███▊      | 2311/6115 [03:44<05:55, 10.69it/s]

In [ ]:
Data = pd.read_csv(Branch_File)[ColumnNames]
X    = Data.to_numpy()

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=61, random_state=0).fit(X)

DiatData[0]['groups'] = kmeans.labels_
DiatData[0].to_csv(Group_File, index=False)